In [1]:
# Importing the required libraries
import pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from functools import partial

In [ ]:
from dotenv import load_dotenv

from pathlib import Path

env_path = Path("../../.env-live")

if env_path.exists():
    print('envs Loaded')
    load_dotenv(dotenv_path=env_path)
from jrjModelRegistry.jrjModelRegistry import registerAJrjModel

envs Loaded
ac-u4uweop-shard-00-00.oyjqzhs.mongodb.net:27017: timed out (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),ac-u4uweop-shard-00-02.oyjqzhs.mongodb.net:27017: timed out (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),ac-u4uweop-shard-00-01.oyjqzhs.mongodb.net:27017: timed out (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 685c51445e27ea6ecd3585b2, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('ac-u4uweop-shard-00-00.oyjqzhs.mongodb.net', 27017) server_type: Unknown, rtt: None, error=NetworkTimeout('ac-u4uweop-shard-00-00.oyjqzhs.mongodb.net:27017: timed out (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>, <ServerDescription ('ac-u4uweop-shard-00-01.oyjqzhs.mongodb.net', 27017) server_type: Unknown, rtt: None, error=NetworkTimeout('ac-u4uweop-shard-00-01.oyjqzhs.mongodb

In [ ]:
def generalDtPredictor(self, transformedData):
    import pandas as pd
    probs = self.predict_proba(transformedData)
    return pd.Series(probs[:, 1])

In [ ]:
def generalDtTransformer(dataForTransfer = None):
    import pandas as pd
    if isinstance(dataForTransfer, pd.DataFrame):
        df = dataForTransfer.copy()
    else:
        df = pd.DataFrame(dataForTransfer)
    return df

In [ ]:
# Reading the csv file and putting it into 'df' object.
df = pd.read_csv('heart_v2.csv')

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
hDSampleData = {
    "age": [70],
    "sex": [1],
    "BP": [130],
    "cholestrol": [322],
}
hDSampleData

In [ ]:
# Putting feature variable to X
X = df.drop('heart disease',axis=1)

# Putting response variable to y
y = df['heart disease']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

Fitting the decision tree with default hyperparameters, apart from max_depth which is 3 so that we can plot and read the tree.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier(max_depth=3)
dt.fit(generalDtTransformer(X_train), y_train)

In [ ]:
# !pip install six

In [ ]:
# Importing required packages for visualization
from IPython.display import Image  
from six import StringIO
from sklearn.tree import export_graphviz
import pydotplus, graphviz

In [ ]:
# plotting tree with max_depth=3
dot_data = StringIO()  

export_graphviz(dt, out_file=dot_data, filled=True, rounded=True,
                feature_names=X.columns, 
                class_names=['No Disease', "Disease"])

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())
# Image(graph.create_png(),width=800,height=900)
# graph.write_pdf("dt_heartdisease.pdf")

#### Evaluating model performance

In [ ]:
y_train_pred = dt.predict(generalDtTransformer(X_train))
y_test_pred = dt.predict(generalDtTransformer(X_test))
y_test_pred_probs = dt.predict_proba(generalDtTransformer(X_test))

In [ ]:
y_test_pred

In [ ]:
y_test_pred_probs

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
print(accuracy_score(y_train, y_train_pred))
confusion_matrix(y_train, y_train_pred)

In [ ]:
print(accuracy_score(y_test, y_test_pred))
confusion_matrix(y_test, y_test_pred)

In [ ]:
dt.predict_proba(generalDtTransformer(hDSampleData))

In [ ]:
dt.transformer = generalDtTransformer
dt.mainPredictor = partial(generalDtPredictor, dt)

In [ ]:
dt.mainPredictor(dt.transformer(hDSampleData))

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
import json

In [ ]:
dt.get_params()

In [ ]:
y_pred = dt.predict(generalDtTransformer(X_test))
# y_pred
score = accuracy_score(y_test, y_pred)
# score
report = classification_report(y_test, y_pred, output_dict=True)
# report

In [ ]:
decision_tree_metadata = {
    "modelName": "Grace_Aiyedogbon__dTModel1",
    "version": "1.0.1",
    "params": dt.get_params(),  # All model hyperparameters
    "score": float(score),         # Accuracy
    "modelLibrary": "sklearn.tree.DecisionTreeClassifier",
    "libraryMetadata": {
        "feature_importances": dt.feature_importances_.tolist(),
        "n_features": int(dt.n_features_in_),
        "n_classes": int(dt.n_classes_),
        "classes": dt.classes_.tolist(),
        "depth": int(dt.get_depth()),
        "n_leaves": int(dt.get_n_leaves()),
        "classification_report": report
    },
     "sampleData": {
        "dataForTransfer": hDSampleData
    }
}

In [ ]:
registerAJrjModel(
    dt,
    decision_tree_metadata
)